In [3]:
%run ./BaseUPCDetector.ipynb
%run ./Proxy.ipynb
%run ./DUPDetector.ipynb
%run ./DynamicProxyDetector.ipynb
%run ./BytecodeDecompiler.ipynb
%run ./ESUPDetector.ipynb
%run ./GoogleClient.ipynb

from abc import ABC, abstractmethod
from overrides import overrides

class SMUPDetector(BaseUPCDetector):
    @overrides
    def parse_decompiled_bytecode(self, decompiled_bytecode_lines):
        self.prepare_env()
        for curr_loc, curr_line in enumerate(decompiled_bytecode_lines):
            self.parse_assignment_exp(curr_loc, curr_line)
            self.parse_function_exp(curr_loc, curr_line)
            self.map_assignments_to_function(curr_loc, curr_line)
            self.parse_contract_storage(curr_line)
            self.parse_proxy_function(curr_loc, curr_line)
    @overrides
    def is_upc(self, address, decompiled_bytecode, bytecode):
        # parse the decmopiled bytecode to collect storage, function, proxy function, relevant delegatecall informations
        self.parse_decompiled_bytecode(decompiled_bytecode)
        # if no relevant delegatecalls are found
        if len(self.delegatecalls) == 0:
            yield (("",""), 'NON-UPC:NO RELEVANT DELEGATECALL', "", [])
        # else if at least one relevant delegatecall is found
        else:
            self.exclude_forwarder_delegatecalls(bytecode)
            self.exclude_duplicate_delegatecalls()
            
            if len(self.delegatecalls) == 0:
                yield (("",""), 'NON-UPC:FORWARDER PROXY', "", [])

            # print(self.delegatecalls)
            # Here, we check the storage to see if delegatecall is sent to any of storage variables. if yes, we store
            # the variable name and its storage slot.
            for delegatecall in self.delegatecalls:
                self.piv = set() # primary impact variables
                self.siv = set() # secondary impact variables
                self.tiv = set() # teritiary impact variables
                self.qiv = []    # quaternary impact variables
                self.impact_var_slots = set()   
                self.implementation_storage = set()
                self.is_piv_of_type_mapping = False
                self.upgrade_funcs1 = set()
                self.upgrade_funcs2 = set()
                self.all_impact_vars = list()
                
                detector_type = self.set_detector_type(delegatecall)
                
                if detector_type == "ESUP":
                    yield (delegatecall, "ESUP Detector","",[])

                else:
                    self.detect_primary_impact_variables(delegatecall)
                    self.detect_quaternary_impact_variables(delegatecall)
                    self.detect_secondary_impact_variables()
                    self.detect_teritiary_impact_variables()
                    
                    # =====================================
                    # make a set of all possible impact variables that can change the implementatio variable value (a.k.a., implementation contract address)
                    # =======================================================================================================                
                    self.all_impact_variables = list(set(self.piv.union(self.siv).union(self.tiv).union(self.qiv)))
                    
                    if self.DEBUG_MODE:
                        print('primary:', self.piv) 
                        print('secondary:',self.siv)
                        print('tertiary:', self.tiv)
                        print('quaternary:', self.qiv)
                        print('storage:',self.impact_var_slots)
                        print('all imps:',self.all_impact_variables)
                        print()

                    
                    self.detect_upgrade_functions()
                    # if we detect an upgrade function within the proxy then the is an SMUP upgradeability proxy contract
                    if len(self.upgrade_funcs1) > 0 and len(self.upgrade_funcs2) > 0:
                        upgrade_func_to_line = [(func, self.all_func_start_end_line[func]) for func in list(self.upgrade_funcs1) + list(self.upgrade_funcs2)]
                        yield (delegatecall, 'UPC:SMUP:1', upgrade_func_to_line, list(self.impact_var_slots), list(self.state_vars), list(self.implementation_storage), list(self.qiv))
                    elif len(self.upgrade_funcs1) > 0:
                        upgrade_func_to_line = [(func, self.all_func_start_end_line[func]) for func in list(self.upgrade_funcs1)]
                        yield (delegatecall, 'UPC:SMUP:2', upgrade_func_to_line, list(self.impact_var_slots), list(self.state_vars), list(self.implementation_storage), list(self.qiv))
                    elif len(self.upgrade_funcs2) > 0:
                        upgrade_func_to_line = [(func, self.all_func_start_end_line[func]) for func in list(self.upgrade_funcs2)]
                        yield (delegatecall, "UPC:SMUP:3", upgrade_func_to_line, list(self.impact_var_slots), list(self.state_vars), list(self.implementation_storage), list(self.qiv))
                    
                    # =====================================
                    # if we have not found an upgrade function at this point
                    # and we found some storage variables for the delegatecall address but this storage
                    # variables are not upgradeable within the proxy contract, then perhaps they are controlled by an implementation
                    # contract, which in this case they represent a uups pattern.
                    # =======================================================================================================   
                    elif len(self.piv) > 0:                
                        yield (delegatecall, "DUP Detector","", list(self.impact_var_slots), self.is_piv_of_type_mapping)

                    # =====================================
                    # the following condition rar
                    # else if we have not found any storage variables, and the delegatecall address is not upgradeable within the
                    # the proxy contract, the delegatecall address is either controlled by a beacon or uups or anything else.
                    # =======================================================================================================           
                    elif len(self.qiv) > 0:
                        yield (delegatecall, "FAILURE:IMPLEMENTATION VARIABLE NOT FOUND:1","", list(self.impact_var_slots))
                    else:
                        yield (delegatecall, "FAILURE:IMPLEMENTATION VARIABLE NOT FOUND:2","", list(self.impact_var_slots))